# 映像メディア処理特論 課題3
「課題3: 画像キャプションの自動生成」について取り組みました.

## (1) テキスト生成における自動評価手法について

### BLEU

BLEUは2002年に [2] の論文によって提案された. 参照文群 $\mathscr{S}_\mathrm{ref}$ 中の文章 $S_{\mathrm{ref}} \in \mathscr{S}_\mathrm{ref}$ の $N$-gram がどれくらい候補群 $\mathscr{S}_{\mathrm{sys}}$ 中の生成されたテキスト $S_{\mathrm{sys}} \in \mathscr{S}_{\mathrm{sys}}$ に含まれているかという適合率 $p_n$ に基づく指標である.  


$$
\mathrm{BLEU} = \exp{\left(
    \sum_{n = 1}^N w_n \log p_n
\right)} \cdot \mathrm{BP} \\

\begin{align*}
\mathrm{where} \ \\
p_n &= \frac
    {
        \displaystyle
        \sum_{S_{\mathrm{sys}} \in \mathscr{S}_{\mathrm{sys}}}
        \sum_{\mathrm{ngram} \in \mathrm{n\_gram}(S_{\mathrm{sys}})}
        {\mathrm{count}_{\mathrm{clip}}(\mathrm{ngram})}
    }
    {
        \displaystyle
        \sum_{S_{\mathrm{sys}} \in \mathscr{S}_{\mathrm{sys}}}
        \sum_{\mathrm{ngram} \in \mathrm{n\_gram}(S_{\mathrm{sys}})}
        {\mathrm{count}(\mathrm{ngram})}
    } \ \\
\mathrm{BP} &= \exp{\left(
    \min \left(1 - \dfrac{l_{\mathrm{ref}}}{l_{\mathrm{sys}}}, 0 \right)
\right)} \\
\mathrm{count}_{\mathrm{clip}}(\mathrm{ngram}) &= \min(\mathrm{count}(\mathrm{ngram}), \mathrm
{max\_ref\_count})
\end{align*}
$$

$N$ は $N$-gram の最大値であり, $w_n = \frac{1}{n}$ であることが多い.  
$\mathrm{max\_ref\_count}$ は参照テキスト $S_{\mathrm{ref}} \in \mathscr{S}_\mathrm{ref}$ の $N$-gram の出現回数の最大値, $\mathrm{count}(\mathrm{ngram})$ は文章 $C$ での $N$-gram の出現回数を表す.

$BP$ は `brevity penalty` と呼ばれ, 短い生成文程スコアが高くなってしまう問題を防ぐための項で短い文章であればあるほど小さくなり, ペナルティとなる.

#### 具体例
$\mathscr{S}_\mathrm{sys} = \{ S_\mathrm{sys} \}, S_{\mathrm{sys}}$ = `"the the the the the the the"`,  
$S_{\mathrm{ref}}$ = `"The cat is on the mat"`
の時  
$N = 4$ を考える.  
$n = 1$ での適合率 $p_1$ は,

$$
\begin{align*}
\mathrm{ngram} &= \mathrm{[``the", ``the", ``the", ``the", ``the", ``the", ``the"]} \\
\mathrm{count}(\mathrm{ngram}) &= 7 \\
\mathrm{count\_clip}(\mathrm{ngram}, \textrm{``The cat is on the mat"}) &= \min(2, 7) = 2
\end{align*}
$$
より, $p_1 = \frac{2}{7}$ となる. $\mathrm{BP} = 1$ より, $\mathrm{BLEU}_1 = \frac{2}{7}$ となる.   
$2$-gram以上では一致するものが無いので, $p_i = 0 (i = 2, 3, 4)$ となり, $\mathrm{BLEU}_i = 0$ となる.  
よって $\mathrm{BLEU} = \frac{2}{7}$ となる.

### ROUGE

ROUGE-$N$ は再現率を元にした指標で, 参照文章群 $\mathscr{S}_\mathrm{ref}$ に対する生成文 $S_{\mathrm{sys}}$ の ROUGE-$N$ スコアは

$$
\textrm{ROUGE-}N = \frac{
    \displaystyle
    \sum_{S \in \mathscr{S}_{\mathrm{ref}}}
    \sum_{\mathrm{ngram} \in \mathrm{n\_gram}(S)}
    \mathrm{count}_{\mathrm{match}}(\mathrm{ngram})
}{
    \displaystyle
    \sum_{S \in \mathscr{S}_{\mathrm{ref}}}
    \sum_{\mathrm{ngram} \in \mathrm{n\_gram}(S)}
    \mathrm{count}(\mathrm{ngram})
    
} \\
$$

$\mathrm{count}_{\mathrm{match}}(\mathrm{ngram})$ は $S_{\mathrm{sys}}$ と $S_{\mathrm{ref}}$ に共に現れる $N$-gramの個数の最大値を表す.

#### 具体例
$S_{\mathrm{ref}}$ = `"I am a cat"`, $S_{\mathrm{sys}}$ = `"there is a cat on the mat"` のとき,   
ROUGE-$1$ は, $\mathrm{count}(\mathrm{ngram}) = 7$, $1$-gram の `"a"` と `"cat"` が $\mathrm{n\_gram}(S_{\mathrm{sys}})$ にも含まれるので ROUGE-$1$ = $\frac{2}{7}$ となる.  
ROUGE-$2$ は, $\mathrm{count}(\mathrm{ngram}) = 6$, $2$-gram の `"a cat"` が $\mathrm{n\_gram}(S_{\mathrm{sys}})$ にも含まれるので ROUGE-$2$ = $\frac{1}{6}$ となる.  

### ROUGE-L

ROUGE-Lは最長共通部分列(LCS)の長さを元にした指標である.

$$
\textrm{ROUGE-L} = \frac{(1 + \beta^2) R_{\mathrm{lcs}} P_{\mathrm{lcs}}}{R_{\mathrm{lcs}} + \beta^2 P_{\mathrm{lcs}}} \\
\begin{align*}
\mathrm{where} \ 
P_\mathrm{lcs} &= \frac{\mathrm{LCS}(S_\mathrm{sys}, S_\mathrm{ref})}{ |S_\mathrm{ref}| } \\
R_\mathrm{lcs} &= \frac{\mathrm{LCS}(S_\mathrm{sys}, S_\mathrm{ref})}{ |S_\mathrm{sys}| }
\end{align*}
$$

#### 具体例
$\beta = 1$, $S_{\mathrm{ref}}$ = `"I am a cat"`, $S_{\mathrm{sys}}$ = `"there is a cat on the mat"` のとき, $\mathrm{LCS}(S_\mathrm{sys}, S_\mathrm{ref}) = 2$ (`"a cat"`) より,  
$\mathrm{R}_\mathrm{lcs} = \frac{2}{4}, \mathrm{P}_\mathrm{lcs} = \frac{2}{7}$ よって,  
$\textrm{ROUGE-L} = \frac{4}{11}$ となる.

### METEOR

METEORは [4] で提案された評価指標で, BLEU等に比べてより人間が評価した時の指標と相関するように設計された指標である. $1$-gramでの適合率を $P$, 再現率を $R$ とすると, $\mathrm{METEOR}$ は以下のように計算できる.

$$
\textrm{METEOR} = \textrm{Fmean} \cdot (1 - \textrm{Penalty}) \\
\begin{align*}
\mathrm{where} \ 
\mathrm{Fmean} &= \frac{PR}{\alpha P + (1 - \alpha) R} \\
\mathrm{Penalty} &= \gamma \cdot \left(
    \frac{\mathrm{\#chunks}}{\mathrm{\#unigrams\_matched}}
\right)^\beta
\end{align*}
$$

$\textrm{\#unigrams\_matched}$ は共通した単語数を表し, $\textrm{\#chunks}$ は部分列のかたまりの個数を表している.

#### 具体例
$\alpha = 0.9, \beta = 3, \gamma = 0.5$ の時,
$S_{\mathrm{ref}}$ = `"I am a cat"`, $S_{\mathrm{sys}}$ = `"there is a cat on the mat"` のとき,   
$P = \frac{2}{4}, R= \frac{2}{7}$, $F_\mathrm{mean} = \frac{20}{67}$  
$\mathrm{\#unigrams\_matched} = 2$ (`"a"`, `"cat"`), $\mathrm{\#chunks} = 1$ (`"a cat"`) なので $\mathrm{Penalty} = \frac{1}{16}$.  
よって, $\mathrm{METEOR} = \frac{75}{268}$ となる.

### Recall@k

[5] 等で用いられている情報検索の評価指標であり, 入力画像に対して上位 $k$ 件のキャプションを検索した際に正解のキャプションがいくつ含まれているかどうか割合を $\mathrm{Recall@}k$ としている.

#### 具体例
正解キャプションが全体で5つあり, Top-$5$ に正解キャプションが3つ含まれている場合の $\mathrm{Recall@}5 = \frac{3}{5}$ となる.

## (2) 画像キャプションの最新の技術動向について


2015年以降, 深層学習を用いた画像キャプション生成がより盛んになり, キャプション生成の精度をより高めようという潮流と, よりマルチモーダルに多様な課題を解決しようという課題自体の進化の潮流で発展していく.  

### よりキャプションの精度を高めるための研究
Attentionの他に精度向上に寄与したのが損失関数の改善である.  
以前は, キャプションに対して交差エントロピーを損失関数としていたが, 画像からキャプションを生成するタスクとその逆のタスクに対して損失関数を設定して損失を最小化するというような一貫性損失を用いた手法が提案された.  

例えば [7] では, 画像 $I$ と画像内の領域 $r$ からその領域のキャプション文章 $w$ を生成するタスク $G: I \times r \to w$ に対して損失関数 $C_\mathrm{gen}$ を設定し, その逆に 画像 $I$ と質問 $q$ と領域の候補 $\mathcal{R}$ から適切な領域 $\hat{r}$ を選択する認識タスク $C: I \times q \times \mathcal{R} \to r$ に対する損失関数 $C_\mathrm{com}$ を設定し, それらを組み合わせた一貫性損失 $L = L_\mathrm{gen} + \lambda L_\mathrm{com}$ を最小化するように学習することでより精度の高いキャプションを生成することに成功している.  

### キャプション生成の課題自体の進化
また, キャプション生成課題自体の進化についても説明する.  
従来の画像キャプション生成タスクは1枚の画像の全体に対し, 客観的な文章を英語で生成するタスクであったが, キャプションする対象はより多様になり, 前述した [7] では画像内の複数領域に対するキャプションを生成したり, 動画に対してのキャプション生成も行われている.  

また, 英語以外の言語での画像キャプション生成についても検討されている. [8] では, 英語で事前学習済みの画像キャプション生成モデルを用意し, 入力部分に最も近い層以外を未学習の日本語生成キャプション生成モデルにして日本語のデータセットで学習した結果, 日本語のみのデータセットで1から学習させるよりも性能が改善したと報告している. この結果より, 言語間を跨ぐ画像キャプション生成も可能であると言っている.  

また, 画像を用いたマルチモーダルな対話システムについても研究されていて, より高度なマルチモーダルタスクを想定したデータセットが存在する.  
正解画像を見ている相手に対して, いくつかの質問をすることで画像を推定する対話のデータセットや, 目的地までの経路移動を自然言語により視覚的な目印となり得る情報と移動方向を指示したデータセットなどが構築されている.

### 参考文献
[1]
⽶⾕ ⻯, et al. コンピュータビジョン : 広がる要素技術と応⽤… 共⽴出版, 2018.  
[2] 
Papineni, K., Roukos, S., Ward, T., & Zhu, W.J. (2002). Bleu: a Method for Automatic Evaluation of Machine Translation. In Proceedings of the 40th Annual Meeting of the Association for Computational Linguistics (pp. 311–318). Association for Computational Linguistics.  
[3]
Lin, C.Y. (2004). ROUGE: A Package for Automatic Evaluation of Summaries. In Text Summarization Branches Out (pp. 74–81). Association for Computational Linguistics.  
[4]
Banerjee, S., & Lavie, A. (2005). METEOR: An automatic metric for MT evaluation with improved correlation with human judgments. In Proceedings of the acl workshop on intrinsic and extrinsic evaluation measures for machine translation and/or summarization (pp. 65–72).  
[5]
Hodosh, M., Young, P., & Hockenmaier, J. (2013). Framing image description as a ranking task: Data, models and evaluation metrics. Journal of Artificial Intelligence Research, 47, 853–899.  
[6]
牛久祥孝 (2019). 画像に関連した言語生成の取組み. 人工知能, 34(4), 483–491.  
[7]
Luo, R., & Shakhnarovich, G. (2017). Comprehension-guided referring expressions. arXiv:1701.03439 [cs].  
[8]
Miyazaki, T., & Shimizu, N. (2016). Cross-Lingual Image Caption Generation. In Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers) (pp. 1780–1790). Association for Computational Linguistics.  
